In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/DATA QUALITY MANAGEMENT')

In [ ]:
%cd /content/drive/My Drive/Colab Notebooks/DATA QUALITY MANAGEMENT

/content/drive/My Drive/Colab Notebooks/DATA QUALITY MANAGEMENT


In [ ]:
import pandas as pd
import numpy as np
import random
import string
import re

In [26]:
name_dataset            = pd.read_csv('dataset/name_dataset.csv')
name_dataset            = name_dataset[['text','initial_label','data_type']]
name_dataset['label']   = name_dataset['initial_label']
name_dataset

# label 0 = tipe data name value false
# label 1 = tipe data name value true
# label 2 = tipe data email value false
# label 3 = tipe data email value true

,text,initial_label,data_type,label
0,Bkybx Qut Xkszjr,1,name,1
1,Uljtvsgw Btway,1,name,1
2,Mjtcmwrl Olqnmaq,1,name,1
3,Tfz Acvedqw,1,name,1
4,Kczrzb Msjszp Dxe,1,name,1
...,...,...,...,...
1995,;betsauw lncfqpt,1,name,1
1996,;$rae %clhqek pjy,1,name,1
1997,diolkos :;mrrbwx,1,name,1
1998,3=*fsf tdlp,1,name,1


In [27]:
email_dataset = pd.read_csv('dataset/email_dataset.csv')
email_dataset = email_dataset[['text','initial_label','data_type']]
email_dataset['label']  = email_dataset['initial_label'].replace({0:2,1:3})
email_dataset

,text,initial_label,data_type,label
0,p5Ruq@dy.w2.ys.nzw,1,email,3
1,DZ-5@6v1.92.du7d.fe,1,email,3
2,vCqbqV@d0te.qhk,1,email,3
3,J0ECxeIZ-@biyd.13bn.q7t919lcuy.yo,1,email,3
4,yA3FIK5@47k39dhka.rfn,1,email,3
...,...,...,...,...
1995,XBjfaOv<jfaOveP@jsqfbuwek.owm+owm,0,email,2
1996,oy2G+oy2GhS@ufgggxthy.djyzvor?vor.bvfc,0,email,2
1997,cJreOTtvAL@hqsaxya.opre|re,0,email,2
1998,e/tfstzpkR@zctuokuw.djenxayky.p=pub,0,email,2


In [28]:
# dataset = pd.get_dummies(dataset, columns=['data_type'])
from sklearn.preprocessing import LabelEncoder
dataset           = pd.concat([email_dataset, name_dataset], ignore_index=True)
label_encoder     = LabelEncoder()
dataset['data_type']  = label_encoder.fit_transform(dataset['data_type'])

In [29]:
dataset = dataset.drop('initial_label', axis=True)
df = dataset
df

,text,data_type,label
0,p5Ruq@dy.w2.ys.nzw,0,3
1,DZ-5@6v1.92.du7d.fe,0,3
2,vCqbqV@d0te.qhk,0,3
3,J0ECxeIZ-@biyd.13bn.q7t919lcuy.yo,0,3
4,yA3FIK5@47k39dhka.rfn,0,3
...,...,...,...
3995,;betsauw lncfqpt,1,1
3996,;$rae %clhqek pjy,1,1
3997,diolkos :;mrrbwx,1,1
3998,3=*fsf tdlp,1,1


In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding

# Contoh data
data = df

# CountVectorizer dengan char=True
vectorizer = CountVectorizer(analyzer='char', binary=True)
X_text = vectorizer.fit_transform(data['text'])

# Fitur boolean
X_boolean = data[['data_type_email', 'data_type_name']].astype(int).values

# Gabungkan fitur menjadi satu matriks fitur
X = np.hstack((X_text.toarray(), X_boolean))

# Kolom yang diprediksi (multikelas)
y = data['data_type'].values

# Split data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Jumlah kelas
num_classes = len(np.unique(y))

# Buat model LSTM
model = Sequential()
model.add(Embedding(X.shape[1], 128, input_length=X.shape[1]))
model.add(LSTM(64))
model.add(Dense(num_classes, activation='softmax'))

# Kompilasi dan pelatihan model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, pd.get_dummies(y_train), epochs=15, batch_size=16)

# Evaluasi model pada data uji
loss, accuracy = model.evaluate(X_test, pd.get_dummies(y_test))
print('Loss:', loss)
print('Accuracy:', accuracy)

In [13]:
#Save models dan feature vector to directory

import pickle

model.save("model/general_validation_lstm_model.keras")

with open('model/general_validation_lstm_vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

In [15]:
def generate_prediction(label):
    rule_dict = {
        0: "Tipe data name dengan value false",
        1: "Tipe data name dengan value true",
        2: "Tipe data email dengan value false",
        3: "Tipe data email dengan value true"
    }
    return rule_dict.get(label, "Label tidak dikenali")

In [18]:
# Assuming you have trained and compiled the model as shown in the previous code

# Example input data for prediction
input_data = pd.DataFrame({
    'text': ['masharry.net@gmail.com', 'Juan &cesar'],
    'data_type_email': [True, False],
    'data_type_name': [False, True]
})

# Transform the input data into the same format as the training data
X_text_pred = vectorizer.transform(input_data['text'])
X_boolean_pred = input_data[['data_type_email', 'data_type_name']].astype(int).values
X_pred = np.hstack((X_text_pred.toarray(), X_boolean_pred))

# Make predictions
predictions = model.predict(X_pred)

# Convert predictions to class labels
predicted_labels = np.argmax(predictions, axis=1)

print("Predictions:")
for prediction in predicted_labels:
    prediction_description = generate_prediction(prediction)
    print("Label:", prediction, "-", prediction_description)

1/1 [==============================] - 0s 26ms/step
Predictions:
Label: 1 - Tipe data name dengan value true
Label: 0 - Tipe data name dengan value false


In [19]:
#Using Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocessing teks
vectorizer = CountVectorizer(max_features=1000, analyzer='char')
X_text = vectorizer.fit_transform(df['text']).toarray()

# Input untuk kolom data_type_email dan data_type_name
X_boolean = df[['data_type_email', 'data_type_name']].astype(int).values

# Gabungkan kedua input
X = np.concatenate((X_text, X_boolean), axis=1)

# Target
Y = df['label'].values

# Train-test split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

# Train logistic regression model
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)

# Predictions on training and validation set
Y_train_pred = logreg.predict(X_train)
Y_test_pred = logreg.predict(X_test)

# Calculate accuracy
train_accuracy = accuracy_score(Y_train, Y_train_pred)
test_accuracy = accuracy_score(Y_test, Y_test_pred)
print("Training Accuracy:", train_accuracy)
print("Validation Accuracy:", test_accuracy)

# Classification report
print("Classification Report:")
print(classification_report(Y_test, Y_test_pred))

Training Accuracy: 0.9896428571428572
Validation Accuracy: 0.9808333333333333
Classification Report:
              precision    recall  f1-score   support

           1       1.00      1.00      1.00       578
           2       0.96      0.97      0.96       307
           3       0.97      0.96      0.96       315

    accuracy                           0.98      1200
   macro avg       0.98      0.98      0.98      1200
weighted avg       0.98      0.98      0.98      1200



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [20]:
#Save Logistic Regression Models
from joblib import dump
dump(logreg, 'model/general_validation_logistic_regression_model.joblib')

with open('model/general_validation_logistic_regression.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

In [21]:
#Prediction using Logistic Regression Model
from joblib import load

# Load LR Model
logreg = load('model/general_validation_logistic_regression_model.joblib')

with open('model/general_validation_logistic_regression.pkl', 'rb') as f:
    vectorizer = pickle.load(f)

# Example input data for prediction
input_data = pd.DataFrame({
    'text': ['masharry.net@gmail.com', '97686'],
    'data_type_email': [True, False],
    'data_type_name': [False, True]
})

# Preprocess the text in 'new_data'
X_text_new = vectorizer.transform(input_data['text']).toarray()

# Preprocess the 'data_type_email' and 'data_type_name' columns
X_boolean_new = input_data[['data_type_email', 'data_type_name']].astype(int).values

# Combine the preprocessed text and boolean features
X_new = np.concatenate((X_text_new, X_boolean_new), axis=1)

# Make predictions using the trained model
predictions = logreg.predict(X_new)

# Display the predictions
print("Predictions:")
for prediction in predictions:
    prediction_description = generate_prediction(prediction)
    print("Label:", prediction, "-", prediction_description)


Predictions:
Label: 2 - Tipe data email dengan value false
Label: 3 - Tipe data email dengan value true
